In [ ]:
pip install gensim
pip install numpy
pip install nltk

In [183]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk

# Takes train data array and predicts which was close to new text

def trainandpredict(newdata, traindata):
    try:
        # Tokenizing the traindata
        tokenized_data = [word_tokenize(document.lower()) for document in traindata]
        
        # Creating TaggedDocument objects
        tagged_data = [TaggedDocument(words=words, tags=[str(idx)]) for idx, words in enumerate(tokenized_data)]
        
        # Training the Doc2Vec model
        model = Doc2Vec(vector_size=100, window=2, min_count=1, workers=4, epochs=1000)
        model.build_vocab(tagged_data)
        model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
        
        # Infer vector for a new document
        inferred_vector = model.infer_vector(word_tokenize(newdata.lower()))
        # print(newdata, "newdata")
        
        # Find most similar documents
        similar_documents = model.dv.most_similar([inferred_vector], topn=len(model.dv))
        
        # Print the most similar documents
        related = []
        for index, score in similar_documents:
            # print(f"Document {index}: Similarity Score: {score}")
            if score>=0.4:
                related.append(index)
        
        return related
    except:
        print("new data:",newdata,"traindata:",traindata)
        return []
     


In [184]:
def get_related_source(response,sources):
    contexts=[]
    for source in sources:
        contexts.append(source['context'])
   
    related=trainandpredict(response,contexts)
    return related

In [185]:
import requests
def citationsperpage(url,page):
    response = requests.get(url)
    data = response.json()
    # print(data["data"])
    citationsperpage=[]
    for index,obj in enumerate(data["data"]["data"]): 
        response=obj["response"]
        source=obj["source"]
        relatedtosource=get_related_source(response,source)
        for k in relatedtosource:
            link=source[int(k)]["link"]
            
            citationsperpage.append({"page":page,"index":index,"link":link , "id": source[int(k)]["id"]})
    
    return citationsperpage


In [186]:
all_citations=[]
#for which pages we want to predict decides based on startpage and stoppage
startpage=1
endpage=2
for page in range(startpage,endpage):
    page_url=f"https://devapi.beyondchats.com/api/get_message_with_sources?page={page}"
    cp=citationsperpage(page_url,page)
    all_citations.append(cp)
    print(page,"completed")
    

In [187]:
all_citations

[[{'page': 0, 'index': 0, 'link': '', 'id': '75'},
  {'page': 0,
   'index': 3,
   'link': 'https://www.drmalpani.com/knowledge-center/articles/pus-cells-in-the-semen---and-how-to-make-sense-of-the-report',
   'id': '4097'},
  {'page': 0,
   'index': 3,
   'link': 'https://www.drmalpani.com/knowledge-center/articles/pus-cells-in-the-semen---and-how-to-make-sense-of-the-report',
   'id': '2424'},
  {'page': 0,
   'index': 4,
   'link': 'https://www.drmalpani.com/knowledge-center/articles/pus-cells-in-the-semen---and-how-to-make-sense-of-the-report',
   'id': '4097'},
  {'page': 0, 'index': 5, 'link': '', 'id': '16990'},
  {'page': 0, 'index': 5, 'link': '', 'id': '437'},
  {'page': 0,
   'index': 5,
   'link': 'https://www.indianangel.in/when-i-first-started-investing-in-startups-we-would-take/3830/',
   'id': '9132'},
  {'page': 0,
   'index': 5,
   'link': 'https://www.indianangel.in/when-i-first-started-investing-in-startups-we-would-take-2/3831/',
   'id': '9133'},
  {'page': 0,
   

from the above based on page no ,index  we can see the output of particular response
example:{'page': 0, 'index': 0, 'link': '', 'id': '75'}
these is page zero with index 0 (for first response) we can see id and links 

## future improvement
## use pandas framework to clear /r/n/ in source 
## use chatgpt client.getchatcompletetions api to get similarity score and get url based on highest similarty(not that much accurate)


# Reframing Problem : combine all responses and combine all sources and train once  then our model works faster than above 